In [1]:
import mxnet as mx
from mxnet import nd
from mxnet import gluon

In [2]:
from mxnet.gluon import nn

net = nn.Sequential()
drop_prob1 = 0.2
drop_prob2 = 0.95

# with net.name_scope():
#     net.add(nn.Flatten())
#     net.add(nn.Dense(256, activation="relu"))
#     net.add(nn.Dropout(drop_prob1))
#     net.add(nn.Dense(256, activation="relu"))
#     net.add(nn.Dropout(drop_prob2))
#     net.add(nn.Dense(10))
# net.initialize()

class MLP(gluon.nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.hidden = gluon.nn.Dense(256, activation='relu')
            self.output = gluon.nn.Dense(10)
    
    def forward(self, x):
        return self.output(self.hidden(x))

class NestMLP(nn.Block):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.denses = [nn.Dense(64, activation='relu'), nn.Dense(32, activation='relu'), nn.Dense(16)]
#         self.net = nn.Sequential()
#         with self.name_scope():
#             self.net.add(nn.Dense(64, activation='relu'))
#             self.net.add(nn.Dense(32, activation='relu'))
#             self.dense = nn.Dense(16, activation='relu')
        for layer in self.denses:
            layer.initialize()
    def forward(self, x):
        _output = x
        for layer in self.denses:
            _output = layer(_output)
#         _output = self.dense(self.net(x))
        return _output

net = nn.Sequential()
net.add(NestMLP())
last_layer = nn.Dense(10)
last_layer.initialize()
net.add(last_layer)
net.collect_params()

/Users/jiananwei/anaconda3/envs/ML/lib/python3.6/site-packages/mxnet/gluon/block.py:228: UserWarning: "NestMLP.denses" is a container with Blocks. Note that Blocks inside the list, tuple or dict will not be registered automatically. Make sure to register them using register_child() or switching to nn.Sequential/nn.HybridSequential instead. 
  .format(name=self.__class__.__name__ + "." + k))


sequential1_ (
  Parameter dense3_weight (shape=(10, 0), dtype=<class 'numpy.float32'>)
  Parameter dense3_bias (shape=(10,), dtype=<class 'numpy.float32'>)
)

In [3]:
import utils
from mxnet import nd
from mxnet import autograd
from mxnet import gluon

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.1})

for epoch in range(100):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))

/Users/jiananwei/anaconda3/envs/ML/lib/python3.6/site-packages/mxnet/gluon/block.py:228: UserWarning: "NestMLP.denses" is a container with Blocks. Note that Blocks inside the list, tuple or dict will not be registered automatically. Make sure to register them using register_child() or switching to nn.Sequential/nn.HybridSequential instead. 
  .format(name=self.__class__.__name__ + "." + k))


Epoch 0. Loss: 2.302321, Train acc 0.104450, Test acc 0.105268
Epoch 1. Loss: 2.301261, Train acc 0.126820, Test acc 0.166867
Epoch 2. Loss: 2.300251, Train acc 0.160373, Test acc 0.099960
Epoch 3. Loss: 2.299223, Train acc 0.134148, Test acc 0.207933
Epoch 4. Loss: 2.298178, Train acc 0.188852, Test acc 0.219351
Epoch 5. Loss: 2.297173, Train acc 0.206798, Test acc 0.197216
Epoch 6. Loss: 2.296129, Train acc 0.234058, Test acc 0.287560
Epoch 7. Loss: 2.295120, Train acc 0.212957, Test acc 0.326723
Epoch 8. Loss: 2.294103, Train acc 0.297693, Test acc 0.228165
Epoch 9. Loss: 2.293066, Train acc 0.217632, Test acc 0.195012
Epoch 10. Loss: 2.292077, Train acc 0.254674, Test acc 0.325220
Epoch 11. Loss: 2.291045, Train acc 0.273020, Test acc 0.273538
Epoch 12. Loss: 2.290058, Train acc 0.288144, Test acc 0.349459


KeyboardInterrupt: 

In [ ]:
weight = net.collect_params()['dense35_weight']
weight.data()








In [ ]:
params.keys()





In [ ]:
net.collect_params()


